# Day 1: The Tyranny of the Rocket Equation

In [1]:
import os

In [2]:
filename = os.path.join(os.getcwd(),"d1_input")
with open(filename) as f:
    mass_inputs = [int(mass) for mass in f.readlines()]

In [3]:
def requiredFuel(mass):
    return mass // 3 -2

def totalFuel(mass):
    input_fuel = requiredFuel(mass)
    f = requiredFuel(input_fuel)
    total_fuel = input_fuel
    while f>0:
        total_fuel += f
        f = requiredFuel(f)
    return total_fuel

def fuelSum(mass_inputs,callback):
    return sum(map(callback,mass_inputs))

In [4]:
# Answer 1
fuelSum(mass_inputs,requiredFuel)

3405637

In [5]:
# Answer 2
fuelSum(mass_inputs,totalFuel)

5105597